#### Individual LSTM-SA

In [21]:
selected_stations = list(camels_data.summary_data[camels_data.summary_data['state_outlet'] == 'SA'].index)

In [ ]:
# Enter the model you wish to run or multiple as per requirements. The models can be accessed through the following names:
#['multi-LSTM', 'multi-linear','multi-CNN', 'multi-Bidirectional-LSTM']

combined= []
for i in range(0,30):
    print('RUN', i)
    input_widths = [5]
    label_widths = [5]
    models = ['multi-LSTM']
    variables = [['streamflow_MLd_inclInfilled', 'precipitation_deficit', 'year_sin', 'year_cos', 'tmax_AWAP', 'tmin_AWAP']]

    permutations_base = list(itertools.product(*[input_widths, label_widths, selected_stations, models, variables]))

    results_baseModels_variables = []
    models_baseModels_variables = []
    errors_baseModels_variables = []

    for input_width, label_width, station, model_name, variable in permutations_base:
        if input_width < label_width:
            continue

        train_df, test_df = camels_data.get_train_val_test(source=variable, stations=selected_stations)

        try:
            print('input_width:{}, label_width:{}, station:{}, model:{}, variables:{}'.format(input_width, label_width, station, model_name, variable))
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print(current_time)

            window = WindowGenerator(input_width=input_width,
                                     label_width=label_width,
                                     shift=label_width,
                                     train_df=train_df,
                                     test_df=test_df,
                                     station=station,
                                     label_columns=['streamflow_MLd_inclInfilled'])

            model = Base_Model(model_name=model_name, window=window, CONV_WIDTH=label_width)

            results_baseModels_variables.append(model.summary())

            pd.DataFrame(results_baseModels_variables).to_csv('results_files/results_ensemble_all_1.csv')

        except:
            errors_baseModels_variables.append([input_width, label_width, station, model])

    Individual_SA= pd.DataFrame(results_baseModels_variables)
    Individual_SA= Individual_SA.mean()
    Individual_SA= Individual_SA.to_dict()
    combined.append(Individual_SA)

In [23]:
Individual_SA_lstm = pd.DataFrame(combined)
Individual_SA_lstm.to_csv('Individual_SA_lstm.csv')

#### Batch-Indicator LSTM- SA 

In [20]:
selected_stations = list(camels_data.summary_data[camels_data.summary_data['state_outlet'] == 'SA'].index)

In [ ]:
combined= []
for i in range(0,30):
    print('RUN',i)
    input_widths = [5]
    label_widths = [5]
    models = ['multi-LSTM']
    variables = [['streamflow_MLd_inclInfilled', 'precipitation_deficit', 'year_sin', 'year_cos', 'tmax_AWAP', 'tmin_AWAP']]

    permutations_base_batch = list(itertools.product(*[input_widths, label_widths, models, variables]))

    results_baseModels_batch = []
    errors_baseModels_batch = []

    for input_width, label_width, model_name, variable in permutations_base_batch:
        try:
            if input_width < label_width:
                continue

            train_df, test_df = camels_data.get_train_val_test(source=variable, stations=selected_stations, discard=0.5)
            multi_window = MultiWindow(input_width=input_width,
                                       label_width=label_width,
                                       shift=label_width,
                                       train_df=train_df,
                                       test_df=test_df,
                                       stations=selected_stations,
                                       label_columns=['streamflow_MLd_inclInfilled'])

            model = Base_Model(model_name=model_name, window=multi_window, CONV_WIDTH=label_width)

            print('input_width:{}, label_width:{}, model:{}, variables:{}'.format(input_width, label_width, model_name, variable))
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print(current_time)

            for station in selected_stations:
                results_baseModels_batch.append(model.summary(station=station))

            pd.DataFrame(results_baseModels_batch).to_csv('results_files/results_batch_all_1.csv')
        except:
            errors_baseModels_batch.append([input_width, label_width, model_name]) 
    Batch_SA= pd.DataFrame(results_baseModels_batch)
    Batch_SA= Batch_SA.mean()
    Batch_SA= Batch_SA.to_dict()
    combined.append(Batch_SA)
 

In [25]:
Batch_SA_LSTM = pd.DataFrame(combined)
Batch_SA_LSTM.to_csv('Batch_SA_LSTM.csv')

#### Batch-Static LSTM SA

In [26]:
selected_stations = list(camels_data.summary_data[camels_data.summary_data['state_outlet'] == 'SA'].index)

In [ ]:
combined=[]
for i in range(0,30):
    print('RUN', i)
    input_widths = [5]
    label_widths = [5]
    models = ['multi-LSTM']
    variables = [['streamflow_MLd_inclInfilled', 'precipitation_deficit', 'year_sin', 'year_cos', 'tmax_AWAP', 'tmin_AWAP', 'q_mean', 'stream_elas', 'runoff_ratio', 'high_q_freq', 'high_q_dur', 'low_q_freq', 'zero_q_freq']]

    permutations_base_batch = list(itertools.product(*[input_widths, label_widths, models, variables]))

    results_baseModels_batch = []
    errors_baseModels_batch = []

    for input_width, label_width, model_name, variable in permutations_base_batch:
        try:
            if input_width < label_width:
                continue

            train_df, test_df = camels_data.get_train_val_test(source=variable, stations=selected_stations)
            multi_window = MultiWindow(input_width=input_width,
                                       label_width=label_width,
                                       shift=label_width,
                                       train_df=train_df,
                                       test_df=test_df,
                                       stations=selected_stations,
                                       label_columns=['streamflow_MLd_inclInfilled'])

            model = Base_Model(model_name=model_name, window=multi_window, CONV_WIDTH=label_width)

            print('input_width:{}, label_width:{}, model:{}, variables:{}'.format(input_width, label_width, model_name, variable))
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print(current_time)

            for station in selected_stations:
                results_baseModels_batch.append(model.summary(station=station))

            pd.DataFrame(results_baseModels_batch).to_csv('results_files/results_batch_static_all_1.csv')
        except:
            errors_baseModels_batch.append([input_width, label_width, model])
    Batch_SA= pd.DataFrame(results_baseModels_batch)
    Batch_SA= Batch_SA.mean()
    Batch_SA= Batch_SA.to_dict()
    combined.append(Batch_SA)

In [30]:
Batchstatic_SA_LSTM = pd.DataFrame(combined)
Batchstatic_SA_LSTM.to_csv('BatchStatic_SA_LSTM.csv')

#### Ensemble Model LSTM- SA

In [23]:
selected_stations = list(camels_data.summary_data[camels_data.summary_data['state_outlet'] == 'SA'].index)

In [24]:
len(selected_stations)

9

In [ ]:
combined=[]
for i in range(0,30):
    print('RUN:', i)
    results_ensemble = []
    input_widths = [5]
    label_widths = [5]

    permutations_ensemble = list(itertools.product(*[input_widths, label_widths]))

    for input_width, label_width in permutations_ensemble:
        np_window = MultiNumpyWindow(input_width=input_width, 
                                     label_width=label_width,
                                     shift=label_width,
                                     timeseries_source=['streamflow_MLd_inclInfilled', 'precipitation_deficit', 'year_sin', 'year_cos', 'tmax_AWAP', 'tmin_AWAP'],
                                     summary_source=['q_mean', 'stream_elas', 'runoff_ratio', 'high_q_freq', 'high_q_dur', 'low_q_freq', 'zero_q_freq'],
                                     summary_data=camels_data.summary_data,
                                     stations=selected_stations,
                                     label_columns=['streamflow_MLd_inclInfilled'])

        ensemble_model = Ensemble_Static(np_window)
        ensemble_model.train()
        print('done')

        for station in selected_stations:
            results_ensemble.append(ensemble_model.summary(station))
                
    Ensemble_SA= pd.DataFrame(results_ensemble)
    Ensemble_SA= Ensemble_SA.mean()
    Ensemble_SA= Ensemble_SA.to_dict()
    combined.append(Ensemble_SA)

In [28]:
Ensemble_SA_LSTM = pd.DataFrame(combined)
Ensemble_SA_LSTM.to_csv('Ensemble_SA_LSTM.csv')